# My code for computing minbounce

In [7]:
# Load the parking function class
load("parking_functions.py")

# Function that generates all possible (n,nk)-Dyck paths
def genfun_Dyck(n,k):
    # The list has +k for a NORTH step and -1 for an EAST step
    list_of_paths = [[k]]
    for length in range(n*k + n - 1):                               # Iterate on the lenght of the lists, add one step
        new_list = []
        for path in list_of_paths:
            if path.count(k) < n:
                new_list = new_list + [path + [k]]     # Add an NORTH step
            if sum(path) > 0:
                new_list = new_list + [path + [-1]]    # Add an EAST step
        list_of_paths = new_list
    # Translate all k's to +1
    transl_list = []
    for path in list_of_paths:
        for i in range(n*k + n):
            if path[i] == k:
                path[i] = 0
        area_path = [n*k + sum(path[0:i]) for i in range(n*k + n) if path[i] == 0]
        transl_list = transl_list + [area_path]
    # Translate to area
    return transl_list

# Compute the bounce for k=1
def bounce(path):
    n = len(path)
    bounce = 0
    x = 0
    iterat = 0
    while x < n:
        x_new = sum([1 for v in path if v >= n - x])
        bounce += iterat*(x_new-x)
        x = x_new
        iterat += 1
    return bounce

# Checks if a path is Dyck
def ok_path(n,k,path):
    for i in range(n-1):                        # Checks decreasing
        if path[i] < path[i+1]:
            return False
    for i in range(n):                          # Checks values on each row
        if path[i] > n*k or path[i] < k*(n-i):      
            return False
    return True

# Compute all decompositions recursively to get min_bounce
def min_bounce(n,k):
    # The result is given by:
    #       dictionary indexed on n,nk - dyck paths (TUPLES)
    #       associated to a list where:
    #           [0]:    minbounce value
    #           [1]:    list of all possible decompositions
    Dk = genfun_Dyck(n,k)       # Compute all dyck paths
    result = {}
    if k == 1:                      # Base      case k = 1
        for path in Dk:
            result = result | { tuple(path) : [ bounce(path), [[tuple(path)]] ] }
    else:                           # Recursive case k > 1
        res_one = min_bounce(n,1)           # Compute k = 1
        res_prev = min_bounce(n,k-1)        # Compute previous
        Dk_prev = list(res_prev.keys())
        for path in Dk:                     # Loop through all new paths
            minbounce = binomial(n,2)*k
            possible_dec = []
            for prev in Dk_prev:            # Loop through old paths
                new_part = [path[i] - prev[i] for i in range(n)]
                if ok_path(n,1,new_part):                                       # If it is a valid decomposition...
                    if res_prev[prev][0] + res_one[tuple(new_part)][0] < minbounce:        #   ...and new minbounce
                        possible_dec = [sorted(dec + [tuple(new_part)]) for dec in res_prev[prev][1]]
                        minbounce = res_prev[prev][0] + res_one[tuple(new_part)][0]
                    else:
                        if res_prev[prev][0] + res_one[tuple(new_part)][0] == minbounce:   #   ...and same minbounce
                            new_dec_list = [sorted(dec + [tuple(new_part)]) for dec in res_prev[prev][1]]    # compute new decompositions
                            for new_dec in new_dec_list:                                                     # remove redundants
                                if new_dec not in possible_dec:
                                    possible_dec = possible_dec + [new_dec]
            result = result | {tuple(path) : [minbounce, possible_dec]}
    return result

Test if the pmaj contributes are preserved

In [26]:
n = 4
k = 5

label = [i+1 for i in range(n)]         # Define the standard labelling for Dyck paths

DPk = genfun_Dyck(n,k)                  # Compute all nk Dyck
DPsq = genfun_Dyck(n,1)                 # Compute all n Dyck

decompositions = min_bounce(n,k)        # Compute min_bounce and decompositions

for path in DPk:                        # Loop over all paths 
    pf_path = ParkFunc(n,n*k,w_area=path,w_label=label)                       # Convert to parking function
    [pmaj_bounce, contributes, wpmaj] = pf_path.pmaj(infos=True)            # Compute pmaj (i.e. k-bounce)

    print("Testing Dyck path: {}".format(path))
    check = False
    for decomp in decompositions[tuple(path)][1]:                              # Run over possible minimal decompositions and look if contributes sum well...
        contr_list = []             
        for factor in decomp:                                               # Compute contributes of decomposition
            contr_factor = []
            pf_factor = ParkFunc(n,n,w_area=list(factor),w_label=label)
            [pmaj_bounce, contr_factor, wpmaj_factor] = pf_factor.pmaj(infos=True)
            contr_list = contr_list + [contr_factor]
        contr_decomp = {i+1:sum([c[i+1] for c in contr_list]) for i in range(n)}# Sum contributes
        if contributes == contr_decomp:                                             # Contributes MATCH!
            #print("\tContributes match using decomposition {}".format(decomp))
            check = True
        else:
            print("\tDecomposition {} does not work:\n\t\tcontributes = {}\n\t\tcontr_decomp = {}".format(decomp, contributes, contr_decomp))

    if not check:                                                                   # No decomposition match...
            print("\tNO suitable decomposition found...")




Testing Dyck path: [20, 20, 20, 20]
Testing Dyck path: [20, 20, 20, 19]
Testing Dyck path: [20, 20, 20, 18]
Testing Dyck path: [20, 20, 20, 17]
Testing Dyck path: [20, 20, 20, 16]
Testing Dyck path: [20, 20, 20, 15]
Testing Dyck path: [20, 20, 20, 14]
Testing Dyck path: [20, 20, 20, 13]
Testing Dyck path: [20, 20, 20, 12]
Testing Dyck path: [20, 20, 20, 11]
Testing Dyck path: [20, 20, 20, 10]
Testing Dyck path: [20, 20, 20, 9]
Testing Dyck path: [20, 20, 20, 8]
Testing Dyck path: [20, 20, 20, 7]
Testing Dyck path: [20, 20, 20, 6]
Testing Dyck path: [20, 20, 20, 5]
Testing Dyck path: [20, 20, 19, 19]
Testing Dyck path: [20, 20, 19, 18]
Testing Dyck path: [20, 20, 19, 17]
Testing Dyck path: [20, 20, 19, 16]
Testing Dyck path: [20, 20, 19, 15]
Testing Dyck path: [20, 20, 19, 14]
Testing Dyck path: [20, 20, 19, 13]
Testing Dyck path: [20, 20, 19, 12]
Testing Dyck path: [20, 20, 19, 11]
Testing Dyck path: [20, 20, 19, 10]
Testing Dyck path: [20, 20, 19, 9]
Testing Dyck path: [20, 20, 19, 8]